In [3]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

import camera_calibration as cc
# make sure to reload the module if it was already imported
import importlib
importlib.reload(cc)

<module 'camera_calibration' from 'c:\\Users\\akoss\\work\\hunbug\\repos\\cam_cv\\src\\camera_calibration.py'>

In [4]:
ocv_cam_calib = cc.OpenCvCalibration((9, 6), 1.0)
NEEDED_SUCCESSFUL_CALIBRATIONS = 10

In [ ]:
# Do calibration on the camera
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

def draw_reprojection(image: np.ndarray, points_3d: np.ndarray, camera_calib_params: cc.CameraCalibParams) -> np.ndarray:
    projected_points = ocv_cam_calib.project_points(points_3d, camera_calib_params)
    result_image = image.copy()
    for i in range(projected_points.shape[0]):
        center = (int(projected_points[i][0][0]), int(projected_points[i][0][1]))
        cv2.circle(result_image, center, 3, (0, 0, 255), -1)
    return result_image

def draw_test_mesh(image: np.ndarray, points_3d: np.ndarray, camera_calib_params: cc.CameraCalibParams,
                    rotation_index: int = 0, connect_points: bool = False) -> np.ndarray:
    # rotation center is the middle of the mesh
    rotation_center = np.mean(points_3d, axis=0)
    ROTATION_ANGLE_PER_FRAME = 0.1
    rotation_angle = ROTATION_ANGLE_PER_FRAME * rotation_index
    rotation_matrix = cv2.Rodrigues(np.array([0, rotation_angle, 0]))[0]
    rotated_points_3d = np.matmul(points_3d - rotation_center, rotation_matrix) + rotation_center
    projected_points = ocv_cam_calib.project_points(rotated_points_3d, camera_calib_params)
    result_image = image.copy()
    for i in range(projected_points.shape[0]):
        if connect_points:
            if i > 0:
                cv2.line(result_image, (int(projected_points[i-1][0][0]), int(projected_points[i-1][0][1])),
                         (int(projected_points[i][0][0]), int(projected_points[i][0][1])), (0, 0, 255), 1)
        else:
            center = (int(projected_points[i][0][0]), int(projected_points[i][0][1]))
            cv2.circle(result_image, center, 3, (0, 0, 255), -1)
    return result_image

def get_heart_mesh_points() -> np.ndarray:
    heart_points = np.array([
        [0,0,0],
        [-2, 2, 0],
        [-1,3,0],
        [0,2,0],
        [1,3,0],
        [2,2,0],
        [0,0,0]
        ], dtype=np.float32)
    return heart_points


# Display function
# ================
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    test_mesh = get_heart_mesh_points()
    while i<=10000:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        try:
            calibration = ocv_cam_calib.calibrate_camera(frame)
            frame = draw_reprojection(frame, ocv_cam_calib.get_3d_points(), calibration)
            frame = draw_test_mesh(frame, test_mesh, calibration, i, True)
        except Exception as e:
            cv2.putText(frame, str(e), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, "Press stop button to stop", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        _, frame = cv2.imencode('.jpeg', frame)
        img = Image(data=frame.tobytes())
        display_handle.update(img)
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)
        i += 1

            
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

Exception in thread Thread-27 (view):
Traceback (most recent call last):
  File "c:\Users\akoss\work\hunbug\repos\cam_cv\.conda\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\akoss\work\hunbug\repos\cam_cv\.conda\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\akoss\AppData\Local\Temp\ipykernel_6888\3058914216.py", line 71, in view
cv2.error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:1113: error: (-215:Assertion failed) !image.empty() in function 'cv::imencode'

